In [ ]:
import json
import pandas as pd
import pathlib
import boto3
from botocore.client import Config
import tomllib

In [3]:
export_file = pathlib.Path(r"C:\code_ibre\load-simpla-annotations\data\ai_assisted\20250813\stand_2025-08-13.json")
image_dir = pathlib.Path(r"C:\code_ibre\load-simpla-annotations\data\ai_assisted\20250813\images")

In [18]:
config = tomllib.load(open("../config.toml", "rb"))

In [ ]:
access_key = config.get("s3-connection").get("access_key")
secret_key = config.get("s3-connection").get("secret_key")
endpoint_url = config.get("s3-connection").get("endpoint_url")

if not all([access_key, secret_key, endpoint_url]):
    raise ValueError("Missing S3 configuration in config.toml")

s3_client = boto3.client(
    "s3",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint_url,
    config=Config(region_name="ch-dk-2",signature_version="s3"),
)

In [6]:
with open(export_file, "r") as f:
    labels = json.load(f)

In [7]:
df_labels = pd.DataFrame(labels)

In [8]:
for data in df_labels["data"]:
    image_url = data.get("image")
    if image_url is not None:
        image_path = pathlib.Path.joinpath(image_dir, image_url.split("/")[-1])
        if image_path.exists():
            continue

        s3_client.download_file(
            Bucket="simpla-gruen-source",
            Key=image_url.split("/")[-1],
            Filename=str(image_path)
        )